In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
import glob
import cv2
from sklearn.model_selection import train_test_split
from keras.layers import Dropout, Dense
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, load_model
from keras.applications import VGG16
from sklearn.metrics import accuracy_score, confusion_matrix

Using TensorFlow backend.


In [2]:
def model(input_shape):
    model = Sequential()
        
    model.add(Dense(512, activation='relu', input_dim = input_shape))
    model.add(Dropout(0.1))
    
    model.add(Dense(256, activation='relu'))
    
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    
    model.add(Dense(64, activation='relu'))
    model.add(Dense(output_dim = no_of_classes, activation='softmax')) 
    
    return model

In [3]:
from keras.models import load_model
no_of_classes = 7
model_top = model(10 * 10 * 512)
EMOTION_DICT = {1: "ANGRY", 2: "DISGUST", 3: "FEAR", 4: "HAPPY", 5: "NEUTRAL", 6: "SAD", 7: "SURPRISE"}
model_VGG = VGG16(weights='imagenet', include_top=False)
model_top.load_weights('model_weights.h5')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=7)`
  del sys.path[0]


In [ ]:
model_top = load_model("model_weights.h5")

In [59]:
def make_prediction(path):
    #converting image to gray scale and save it
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(path, gray)
    print('enter')
    #detect face in image, crop it then resize it then save it
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') 
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 8)
    for (x,y,w,h) in faces:
        face_clip = img[y:y+h, x:x+w]
        new_face_path = path.split('/')
        new_face_path = new_face_path[len(new_face_path)-1]
        new_face_path = new_face_path.rsplit('.',1)
        new_face_path = new_face_path[0] + '-FACE.jpg'
        print(new_face_path)
        if not os.path.exists('faces'):
            os.makedirs('faces')
        cv2.imwrite('faces/'+new_face_path, cv2.resize(face_clip, (350, 350)))
    if faces == ():
        return "No faces detected!"
    #read the processed image then make prediction and display the result
    read_image = cv2.imread('faces/'+new_face_path)
    read_image = read_image.reshape(1, read_image.shape[0], read_image.shape[1], read_image.shape[2])
    read_image_final = read_image/255.0  #normalizing the image
    VGG_Pred = model_VGG.predict(read_image_final)  #creating bottleneck features of image using VGG-16.
    VGG_Pred = VGG_Pred.reshape(1, VGG_Pred.shape[1]*VGG_Pred.shape[2]*VGG_Pred.shape[3])
    top_pred = model_top.predict(VGG_Pred)  #making prediction from our own model.
    emotion_label = top_pred[0].argmax() + 1
    print("Predicted Expression Probabilities")
    print("ANGRY: {}\nDISGUST: {}\nFEAR: {}\nHAPPY: {}\nNEUTRAL: {}\nSAD: {}\nSURPRISE: {}\n\n".format(top_pred[0][0], top_pred[0][1], top_pred[0][2], top_pred[0][3], top_pred[0][4], top_pred[0][5], top_pred[0][6]))
    os.rename('faces/'+new_face_path, 'faces/' + new_face_path.rsplit('.',1)[0] + '-' + str(EMOTION_DICT[emotion_label]) +'.jpg')
    print("Dominant Probability = "+str(EMOTION_DICT[emotion_label])+": "+str(max(top_pred[0])))

In [64]:
import glob
import os

files = glob.glob("../dsc160-midterm-group13/data/scenes/incredibles_example/*.jpg")
for fname in files:
        make_prediction(fname)
        print(fname)

enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-42-1.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-70-3.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-140-2.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-264-2.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-158-3.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-197-1.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-213-3.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-105-1.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-221-1.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-137-3.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-237-1.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-121-3.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-205-3.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-113-1.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-266-3.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-170-1.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-254-1.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-142-3.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-72-2.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-209-3.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-97-3.

../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-65-2.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-98-2.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-108-1.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-133-1.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-225-3.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-101-3.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-217-1.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-193-3.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-278-1.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-31-2.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-176-2.j

../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-272-3.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-199-1.jpg
enter
incredibles-Scene-164-1-FACE.jpg
Predicted Expression Probabilities
ANGRY: 0.9809396266937256
DISGUST: 0.0009875899413600564
FEAR: 0.0005002872785553336
HAPPY: 0.00010224084689980373
NEUTRAL: 0.003196960547938943
SAD: 0.014173716306686401
SURPRISE: 9.962701733456925e-05


Dominant Probability = ANGRY: 0.9809396
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-164-1.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-240-1.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-156-3.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-23-1.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-11-3.jpg
enter
../dsc160-midterm-group13/data/scenes/incredi

...

../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-165-2.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-33-1.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-215-2.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-93-3.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-131-2.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-76-2.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-129-3.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-146-3.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-250-1.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-19-2.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-174-1.jp

incredibles-Scene-84-3-FACE.jpg
Predicted Expression Probabilities
ANGRY: 0.10777559876441956
DISGUST: 0.0008057868108153343
FEAR: 0.04576513171195984
HAPPY: 0.0016854665009304881
NEUTRAL: 0.6485596895217896
SAD: 0.16568192839622498
SURPRISE: 0.029726320877671242


Dominant Probability = NEUTRAL: 0.6485597
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-84-3.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-126-2.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-145-2.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-261-2.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-28-1.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-47-1.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-75-3.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_ex

../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-184-1.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-179-1.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-14-2.jpg
enter
incredibles-Scene-48-1-FACE.jpg
Predicted Expression Probabilities
ANGRY: 0.6032694578170776
DISGUST: 0.005778421647846699
FEAR: 0.014063838869333267
HAPPY: 0.002525507239624858
NEUTRAL: 0.21894395351409912
SAD: 0.15475262701511383
SURPRISE: 0.0006662358646281064


Dominant Probability = ANGRY: 0.60326946
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-48-1.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-201-2.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-87-3.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-6-3.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_exam

../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-148-3.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-127-3.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-231-1.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-4-2.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-78-2.jpg
enter
incredibles-Scene-85-2-FACE.jpg
Predicted Expression Probabilities
ANGRY: 0.0006256331107579172
DISGUST: 3.6977321542508435e-06
FEAR: 0.09026582539081573
HAPPY: 0.00018046435434371233
NEUTRAL: 0.06808865070343018
SAD: 0.007416021078824997
SURPRISE: 0.8334197402000427


Dominant Probability = SURPRISE: 0.83341974
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-85-2.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-115-1.jpg
enter
../dsc160-midterm-group13/data/scenes/incredib

incredibles-Scene-164-3-FACE.jpg
Predicted Expression Probabilities
ANGRY: 0.9651696681976318
DISGUST: 0.0008355916943401098
FEAR: 0.0018582763150334358
HAPPY: 0.00017423840472474694
NEUTRAL: 0.007826777175068855
SAD: 0.02314213290810585
SURPRISE: 0.000993455876596272


Dominant Probability = ANGRY: 0.96516967
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-164-3.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-240-3.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-156-1.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-23-3.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-181-2.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-11-1.jpg
enter
../dsc160-midterm-group13/data/scenes/incredibles_example/incredibles-Scene-258-2.jpg
enter
../dsc160-midterm-group13/data/scenes/incredib